In [129]:
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image, UnidentifiedImageError
from PIL import Image, ImageOps
from tensorflow.keras import layers, models
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
import tensorflow as tf
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [130]:
import os
from PIL import Image, UnidentifiedImageError

# 图片所在的根目录
root_dir = 'trafficsigns_dataset'  # 请替换成实际的路径

# 用于存储按两层标签分隔的图片的嵌套字典
images_by_label = {}

# 用于遍历数据集并加载图像及其标签的函数
def load_images_and_labels(root_dir):
    # 遍历根目录下的每个子目录（第一层）
    for sub_dir in os.listdir(root_dir):
        sub_dir_path = os.path.join(root_dir, sub_dir)
        
        # 如果子目录确实是一个目录
        if os.path.isdir(sub_dir_path):
            # 初始化第一层标签键
            images_by_label[sub_dir] = {}
            
            # 再次遍历该子目录下的文件夹（第二层，具体的标签）
            for label_dir in os.listdir(sub_dir_path):
                label_dir_path = os.path.join(sub_dir_path, label_dir)
                
                # 如果第二层也是一个目录
                if os.path.isdir(label_dir_path):
                    # 初始化第二层标签键
                    images_by_label[sub_dir][label_dir] = []
                    
                    # 遍历第二层目录下的所有图像文件
                    for image_filename in os.listdir(label_dir_path):
                        # 忽略 .DS_Store 文件
                        if image_filename == '.DS_Store':
                            continue
                        image_path = os.path.join(label_dir_path, image_filename)
                        try:
                            images_by_label[sub_dir][label_dir].append(image_path)
                        except UnidentifiedImageError:
                            # 如果无法识别图像，打印出错信息
                            print(f"Cannot identify image file '{image_path}'")

# 载入图像数据和标签
load_images_and_labels(root_dir)
temp = 0
# 打印每个类别的图像数量
for sub_dir, labels in images_by_label.items():
    for label, images in labels.items():
        print(f"{sub_dir}/{label}: {len(images)} images")
        temp += len(images)

print(temp)

diamond/rightofway: 282 images
hex/stop: 43 images
square/laneend: 118 images
square/parking: 276 images
square/continue: 199 images
square/crossing: 95 images
triangle/giveway: 231 images
triangle/warning: 695 images
round/traveldirection: 124 images
round/limitedtraffic: 125 images
round/speed: 316 images
round/roundabout: 98 images
round/noentry: 375 images
round/noparking: 242 images
round/bicycle: 285 images
round/trafficdirective: 195 images
3699


28no Unusual

In [131]:
def get_image_sizes(root_dir):
    sizes = []
    for subdir, dirs, files in os.walk(root_dir):
        for file in files:
            if file.lower().endswith('.png') or file.lower().endswith('.jpg'):
                try:
                    with Image.open(os.path.join(subdir, file)) as img:
                        sizes.append(img.size)
                except (IOError, UnidentifiedImageError):
                    continue
    return sizes

image_sizes = get_image_sizes(root_dir)

sizes_np = np.array(image_sizes)
min_size = sizes_np.min(axis=0)
max_size = sizes_np.max(axis=0)
mean_size = sizes_np.mean(axis=0)
std_dev_size = sizes_np.std(axis=0)
median_size = np.median(sizes_np, axis=0)

min_size, max_size, mean_size, std_dev_size, median_size

(array([28, 28]),
 array([28, 28]),
 array([28., 28.]),
 array([0., 0.]),
 array([28., 28.]))

In [132]:
train_data = {}
test_data = {}
validation_data = {}

for first_level, second_level_dict in images_by_label.items():
    train_data[first_level] = {}
    validation_data[first_level] = {}
    test_data[first_level] = {}
    
    for label, image_paths in second_level_dict.items():
        temp_images, test_images = train_test_split(image_paths, test_size=0.2, random_state=42)
        train_images, val_images = train_test_split(temp_images, test_size=0.2, random_state=42)
        
        # Save split data
        train_data[first_level][label] = train_images
        test_data[first_level][label] = test_images
        validation_data[first_level][label] = val_images


In [133]:
def create_dataframe_for_16_class(data_dict):
    rows = []
    for first_level, second_level_dict in data_dict.items():
        for label, image_paths in second_level_dict.items():
            for path in image_paths:
                rows.append({'ImagePath': path, 'Label': label})
    return pd.DataFrame(rows)

def create_dataframe_for_5_class(data_dict):
    rows = []
    for first_level, second_level_dict in data_dict.items():
        for label, image_paths in second_level_dict.items():
            for path in image_paths:
                rows.append({'ImagePath': path, 'Label': first_level})
    return pd.DataFrame(rows)

train_df = create_dataframe_for_16_class(train_data)
test_df = create_dataframe_for_16_class(test_data)
validation_df = create_dataframe_for_16_class(validation_data)

In [134]:
batch_size = 64

In [135]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='ImagePath',
    y_col='Label',
    color_mode='grayscale',
    target_size=(28, 28),  
    batch_size=batch_size,
    class_mode='categorical',  # 如果是多分类问题
    shuffle=True
)

val_generator = val_datagen.flow_from_dataframe(
    dataframe=validation_df,
    x_col='ImagePath',
    y_col='Label',
    color_mode='grayscale',
    target_size=(28, 28),
    batch_size=batch_size,
    class_mode='categorical',  # 如果是多分类问题
    shuffle=False
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='ImagePath',
    y_col='Label',
    color_mode='grayscale',
    target_size=(28, 28),
    batch_size=batch_size,
    class_mode='categorical',  # 如果是多分类问题
    shuffle=False
)

Found 2358 validated image filenames belonging to 16 classes.
Found 596 validated image filenames belonging to 16 classes.
Found 745 validated image filenames belonging to 16 classes.


## Transfer data to suit scikit-learn

In [136]:
import numpy as np

def get_data_from_generator(generator):
    batches = []
    labels = []
    # 迭代生成器收集数据和标签
    for batch, label in generator:
        batches.append(batch)
        labels.append(label)
        if len(batches) >= len(generator):
            break  # 确保不会无限迭代
    return np.vstack(batches), np.vstack(labels)


X_train, y_train = get_data_from_generator(train_generator)
X_val, y_val = get_data_from_generator(val_generator)
X_test, y_test = get_data_from_generator(test_generator)

## Decision Tree

In [137]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import numpy as np

# 假设数据提取和重塑代码已经运行
X_train, y_train = get_data_from_generator(train_generator)
X_train = X_train.reshape(X_train.shape[0], -1)

X_val, y_val = get_data_from_generator(val_generator)
X_val = X_val.reshape(X_val.shape[0], -1)

n_epochs = 50  # 设置模拟的迭代次数
max_depth_start = 10  # 开始的最大深度
max_depth_end = 200  # 结束的最大深度

for epoch in range(n_epochs):
    # 计算当前epoch的max_depth
    current_max_depth = max_depth_start + (max_depth_end - max_depth_start) * epoch // n_epochs
    
    # 训练决策树模型
    tree_model = DecisionTreeClassifier(max_depth=current_max_depth)
    tree_model.fit(X_train, y_train)
    
    # 在验证集上评估模型
    y_pred_val = tree_model.predict(X_val)
    val_accuracy = accuracy_score(y_val, y_pred_val)
    
    # 打印类似于深度学习训练输出的信息
    print(f"Epoch {epoch + 1}/{n_epochs} - max_depth: {current_max_depth} - val_accuracy: {val_accuracy:.4f}")


Epoch 1/50 - max_depth: 10 - val_accuracy: 0.7651
Epoch 2/50 - max_depth: 13 - val_accuracy: 0.7869
Epoch 3/50 - max_depth: 17 - val_accuracy: 0.8003
Epoch 4/50 - max_depth: 21 - val_accuracy: 0.7852
Epoch 5/50 - max_depth: 25 - val_accuracy: 0.7886
Epoch 6/50 - max_depth: 29 - val_accuracy: 0.7836
Epoch 7/50 - max_depth: 32 - val_accuracy: 0.7936
Epoch 8/50 - max_depth: 36 - val_accuracy: 0.7819
Epoch 9/50 - max_depth: 40 - val_accuracy: 0.7819
Epoch 10/50 - max_depth: 44 - val_accuracy: 0.7903
Epoch 11/50 - max_depth: 48 - val_accuracy: 0.7819
Epoch 12/50 - max_depth: 51 - val_accuracy: 0.7936
Epoch 13/50 - max_depth: 55 - val_accuracy: 0.7886
Epoch 14/50 - max_depth: 59 - val_accuracy: 0.7785
Epoch 15/50 - max_depth: 63 - val_accuracy: 0.7768
Epoch 16/50 - max_depth: 67 - val_accuracy: 0.7836
Epoch 17/50 - max_depth: 70 - val_accuracy: 0.7953
Epoch 18/50 - max_depth: 74 - val_accuracy: 0.7718
Epoch 19/50 - max_depth: 78 - val_accuracy: 0.7836
Epoch 20/50 - max_depth: 82 - val_accura

## Random Forest

In [138]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

n_epochs = 50  # 设置迭代次数
for epoch in range(n_epochs):
    rf_model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=epoch)
    rf_model.fit(X_train, y_train)
    y_pred_val = rf_model.predict(X_val)
    val_accuracy = accuracy_score(y_val, y_pred_val)
print("Validation Accuracy:", accuracy_score(y_val, y_pred_val))

Validation Accuracy: 0.7449664429530202


## SVM

In [139]:
import numpy as np
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

# 假设 y_train 和 y_val 是 one-hot 编码的，形状为 (样本数, 类别数)
y_train = np.argmax(y_train, axis=1)
y_val = np.argmax(y_val, axis=1)

# 初始化并应用标准化
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# 初始化 SVM 模型
svm_model = SVC(kernel='rbf', C=1.0, gamma='auto', max_iter=200)

# 训练 SVM 模型
svm_model.fit(X_train_scaled, y_train)

# 在验证集上评估 SVM 模型
y_pred_val = svm_model.predict(X_val_scaled)
print("Validation Accuracy:", accuracy_score(y_val, y_pred_val))


/Users/tianbai/anaconda3/envs/DL/lib/python3.10/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Validation Accuracy: 0.9379194630872483


## MLP Baseline Model for Multi-class Task


In [140]:
num_classes = 16
input_shape = (28, 28)
baseline_categorical = tf.keras.models.Sequential([
    tf.keras.Input(shape=input_shape),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

In [141]:
baseline_categorical.compile(loss='binary_crossentropy',
                        optimizer=tf.keras.optimizers.Adam(),
                        metrics=['accuracy'])

In [142]:
epochs = 50
history_baseline_categorical = baseline_categorical.fit(
    train_generator,
    validation_data=val_generator,
    epochs=epochs
)

Epoch 1/50


/Users/tianbai/anaconda3/envs/DL/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step - accuracy: 0.2303 - loss: 0.3156 - val_accuracy: 0.5403 - val_loss: 0.1687
Epoch 2/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.6071 - loss: 0.1572 - val_accuracy: 0.7617 - val_loss: 0.1211
Epoch 3/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.7540 - loss: 0.1187 - val_accuracy: 0.8389 - val_loss: 0.0986
Epoch 4/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.8096 - loss: 0.0979 - val_accuracy: 0.8490 - val_loss: 0.0854
Epoch 5/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.8442 - loss: 0.0849 - val_accuracy: 0.8758 - val_loss: 0.0730
Epoch 6/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.8842 - loss: 0.0715 - val_accuracy: 0.8792 - val_loss: 0.0669
Epoch 7/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.9037 - loss: 0.0604 - val_accuracy: 0.9128 - val_loss: 0.0594
Epoch 8/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.9151 - loss: 0.0584 - val_accuracy: 0.9211 - val_loss: 0.

## MLP Baseline Model with difficult for Multi-class Task

In [143]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,  # 旋转范围（角度），可以调整范围，如 20-40
    width_shift_range=0.1,  # 横向平移范围
    height_shift_range=0.1,  # 纵向平移范围
    shear_range=0.1,  # 剪切范围
    zoom_range=0.1,  # 缩放范围
    horizontal_flip=True,  # 水平翻转
)
val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='ImagePath',
    y_col='Label',
    color_mode='grayscale',
    target_size=(28, 28),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

val_generator = val_datagen.flow_from_dataframe(
    dataframe=validation_df,
    x_col='ImagePath',
    y_col='Label',
    color_mode='grayscale',
    target_size=(28, 28),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='ImagePath',
    y_col='Label',
    color_mode='grayscale',
    target_size=(28, 28),
    batch_size=batch_size,
    class_mode='categorical',  
    shuffle=False
)

Found 2358 validated image filenames belonging to 16 classes.
Found 596 validated image filenames belonging to 16 classes.
Found 745 validated image filenames belonging to 16 classes.


In [144]:
num_classes = 16
input_shape = (28, 28)
baseline_categorical = tf.keras.models.Sequential([
    tf.keras.Input(shape=input_shape),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

In [145]:
baseline_categorical.compile(loss='binary_crossentropy',
                        optimizer=tf.keras.optimizers.Adam(),
                        metrics=['accuracy'])

In [146]:
epochs = 50
history_baseline_categorical = baseline_categorical.fit(
    train_generator,
    validation_data=val_generator,
    epochs=epochs
)

Epoch 1/50


/Users/tianbai/anaconda3/envs/DL/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step - accuracy: 0.1757 - loss: 0.3277 - val_accuracy: 0.3289 - val_loss: 0.2124
Epoch 2/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.3031 - loss: 0.2188 - val_accuracy: 0.4698 - val_loss: 0.1860
Epoch 3/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.3694 - loss: 0.2032 - val_accuracy: 0.4446 - val_loss: 0.1756
Epoch 4/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.3915 - loss: 0.1939 - val_accuracy: 0.5621 - val_loss: 0.1596
Epoch 5/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.4407 - loss: 0.1841 - val_accuracy: 0.6074 - val_loss: 0.1469
Epoch 6/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.4357 - loss: 0.1820 - val_accuracy: 0.6191 - val_loss: 0.1434
Epoch 7/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.4985 - loss: 0.1717 - val_accuracy: 0.6577 - val_loss: 0.1381
Epoch 8/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.4976 - loss: 0.1682 - val_accuracy: 0.6946 - val_loss: 0.

## CNN Model for Multi Class

In [147]:
from tensorflow.keras import models, layers

num_classes = 16  # 根据你的任务调整类别数

cnn_model = models.Sequential([
    layers.Input(shape=(28, 28, 1)),  # 灰度图像输入，1 个通道
    layers.Conv2D(32, (3, 3), padding='same'),  # 第一卷积层，32 个滤波器
    layers.BatchNormalization(),  # 添加批归一化
    layers.Activation('relu'),  # 激活层
    layers.MaxPooling2D((2, 2)),  # 最大池化
    layers.Conv2D(64, (3, 3), padding='same'),  # 第二卷积层，64 个滤波器
    layers.BatchNormalization(),  # 添加批归一化
    layers.Activation('relu'),  # 激活层
    layers.MaxPooling2D((2, 2)),  # 最大池化
    layers.Conv2D(128, (3, 3), padding='same'),  # 第三卷积层，128 个滤波器
    layers.BatchNormalization(),  # 添加批归一化
    layers.Activation('relu'),  # 激活层
    layers.MaxPooling2D((2, 2)),  # 最大池化
    layers.Flatten(),  # 将卷积结果展平
    layers.Dense(512),  # 全连接层
    layers.BatchNormalization(),  # 添加批归一化
    layers.Activation('relu'),  # 激活层
    layers.Dense(num_classes, activation='softmax')  # 输出层，使用 softmax 激活
])

cnn_model.compile(
    loss='categorical_crossentropy',  # 使用多分类交叉熵损失
    optimizer=tf.keras.optimizers.Adam(),  # Adam 优化器
    metrics=['accuracy']  # 使用准确率作为指标
)


In [148]:
epochs = 50
history_cnn = cnn_model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=epochs
)


Epoch 1/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 9s 136ms/step - accuracy: 0.4454 - loss: 1.8079 - val_accuracy: 0.0654 - val_loss: 2.6519
Epoch 2/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 48ms/step - accuracy: 0.8235 - loss: 0.5784 - val_accuracy: 0.1208 - val_loss: 2.8052
Epoch 3/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 46ms/step - accuracy: 0.8972 - loss: 0.3371 - val_accuracy: 0.1510 - val_loss: 3.0375
Epoch 4/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 47ms/step - accuracy: 0.9139 - loss: 0.2745 - val_accuracy: 0.2685 - val_loss: 3.2127
Epoch 5/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 47ms/step - accuracy: 0.9333 - loss: 0.2368 - val_accuracy: 0.2064 - val_loss: 3.3986
Epoch 6/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 47ms/step - accuracy: 0.9520 - loss: 0.1776 - val_accuracy: 0.2651 - val_loss: 2.6813
Epoch 7/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 46ms/step - accuracy: 0.9423 - loss: 0.1774 - val_accuracy: 0.4262 - val_loss: 1.6798
Epoch 8/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 47ms/step - accuracy: 0.9643 - loss: 0.1347 - val_accuracy: 0.4279 - 